In [5]:
import tensorflow as tf

# Check for TensorFlow GPU access
# print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")


AttributeError: module 'tensorflow' has no attribute '__version__'

# 04 利用卷积神经网络实现图像风格迁移
## 原理
* 将风格图的风格和内容图的内容进行融合，所生成的图片，在内容上应尽可能地接近内容图，在风格上应该尽可能接近风格图
* 因此需要定义**内容损失函数**和**风格损失函数**，经过加权后作为总的损失函数。
* 实现步骤如下：
    * 随机产生一张图片
    * 在每轮的迭代中，根据总的损失函数调整图片的像素值
    * 经过多轮迭代，得到优化后的图片
## 内容损失函数
* 不能简单通过像素检索内容损失
* 使用CNN将各个卷积层的输出作为图像的内容，以VGG 19为例，其中包含了多个卷积层、池化层，以及最后的全连接层。<br/>
![jupyter](./VGG19.webp)
* 本例中使用conv4_2的输出作为图像内容表示，定义内容损失函数如下：
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>L</mi><mrow><mi>c</mi><mi>o</mi><mi>n</mi><mi>t</mi><mi>e</mi><mi>n</mi><mi>e</mi><mi>t</mi></mrow></msub><mo stretchy="false">(</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>p</mi><mrow></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>x</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mi>l</mi><mo stretchy="false">)</mo><mo>=</mo><mfrac><mn>1</mn><mn>2</mn></mfrac><munder><mo data-mjx-texclass="OP">∑</mo><mrow><mi>i</mi><mo>,</mo><mi>j</mi></mrow></munder><mo stretchy="false">(</mo><msup><mrow><msub><mi>F</mi><mrow><mi>i</mi><mi>j</mi></mrow></msub></mrow><mo>′</mo></msup><mo>−</mo><msup><mrow><msub><mi>P</mi><mrow><mi>i</mi><mi>j</mi></mrow></msub></mrow><mo>′</mo></msup><msup><mo stretchy="false">)</mo><mrow><mn>2</mn></mrow></msup></math>

## 风格损失函数
* 风格是一个很难说清楚的概念，可能是笔触、纹理、结构、布局、用色等等,这里我们使用卷积层各个特征图之间的互相关作为圈像的风格，以conv1_1为例。
    * 共包含64个特征图即feature map，或者说图像的深度、通道的个数
    * 每个特征图都是对上一层输出的一种理解，可以类比成64个人对同一幅画的不同理解
    * 这些人可能分别偏好印象派、现代主义、超现实主义、表现主义等不同风格
    * 当图像是某一种风格时，可能这一部分人很欣赏，但那一部分人不喜欢
    * 当图像是另一种风格时，可能这一部分人不喜欢，但那一部分人很欣赏
    * 64个人之间理解的差异，可以用特征圈的互相关表示，这里使用 $ Gram $ 矩阵计算互相关
    * 不同的风格会导致差异化的互相关结果
* $ Gram $矩阵的计算如下，如果有64个特征圈，那么$ Gram $矩阵的大小便是 $64*64$，第$ i $行第 $ j $ 列的值表示第 $ i $ 个特征图和第 $ j $ 个特征圈之问的互相关，用内积计算。
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msubsup><mi>G</mi><mrow><mi>i</mi><mi>j</mi></mrow><mo>′</mo></msubsup><mo>=</mo><munder><mo data-mjx-texclass="OP">∑</mo><mrow><mi>k</mi></mrow></munder><mrow><msubsup><mi>F</mi><mrow><mi>i</mi><mi>k</mi></mrow><mo>′</mo></msubsup></mrow><mrow><msubsup><mi>F</mi><mrow><mi>i</mi><mi>k</mi></mrow><mo>′</mo></msubsup></mrow></math>

* 风格损失函数定义如下，对多个卷积层的风格表示差异进行加权：
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>E</mi><mrow><mi>l</mi></mrow></msub><mo>=</mo><mfrac><mn>1</mn><mrow><mn>4</mn><msubsup><mi>N</mi><mrow><mi>l</mi></mrow><mrow><mn>2</mn></mrow></msubsup><msubsup><mi>M</mi><mrow><mi>l</mi></mrow><mrow><mn>2</mn></mrow></msubsup></mrow></mfrac><munder><mo data-mjx-texclass="OP">∑</mo><mrow><mi>i</mi><mo>,</mo><mi>j</mi></mrow></munder><mo stretchy="false">(</mo><msubsup><mi>G</mi><mrow><mi>i</mi><mi>j</mi></mrow><mo>′</mo></msubsup><mo>−</mo><msubsup><mi>A</mi><mrow><mi>i</mi><mi>j</mi></mrow><mo>′</mo></msubsup><msup><mo stretchy="false">)</mo><mrow><mn>2</mn></mrow></msup></math>

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>L</mi><mrow><mi>s</mi><mi>t</mi><mi>y</mi><mi>l</mi><mi>e</mi></mrow></msub><mo stretchy="false">(</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>a</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>x</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo stretchy="false">)</mo><mo>=</mo><munderover><mo data-mjx-texclass="OP">∑</mo><mrow><mi>l</mi><mo>=</mo><mn>0</mn></mrow><mrow><mi>L</mi></mrow></munderover><msub><mi>ω</mi><mrow><mi>l</mi></mrow></msub><msub><mi>E</mi><mrow><mi>l</mi></mrow></msub></math>

* 这里我们使用$ conv1_1 $、$ conv2_1 $、$ conv3_1 $、$ conv4_1 $、$ conv5_1 $ 五个卷积层进行风格损失函数的计算，不同的权重将会导致不同的迁移效果。（约到后面的层越抽象）
## 总的损失函数
* 总的损失函数即内容损失函数和风格损失函数的加权，不同的权重将导致不同的迁移效果。
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mi>L</mi><mrow><mi>t</mi><mi>o</mi><mi>t</mi><mi>a</mi><mi>l</mi></mrow></msub><mo stretchy="false">(</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>p</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>a</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>x</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo stretchy="false">)</mo><mo>=</mo><mi>α</mi><msub><mi>L</mi><mrow><mi>c</mi><mi>o</mi><mi>n</mi><mi>t</mi><mi>e</mi><mi>n</mi><mi>t</mi></mrow></msub><mo stretchy="false">(</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>p</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>x</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo stretchy="false">)</mo><mo>+</mo><mi>β</mi><msub><mi>L</mi><mrow><mi>s</mi><mi>t</mi><mi>y</mi><mi>l</mi><mi>e</mi></mrow></msub><mo stretchy="false">(</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>a</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo>,</mo><mrow data-mjx-texclass="ORD"><mrow data-mjx-texclass="REL"><mover><mi>x</mi><mrow><mpadded height="-3pt" depth="+3pt" voffset="-3pt"><mstyle displaystyle="false" scriptlevel="0"><mrow><mstyle displaystyle="false" scriptlevel="2"><mo stretchy="false">⇀</mo></mstyle></mrow></mstyle></mpadded></mrow></mover></mrow></mrow><mo stretchy="false">)</mo></math>

## Scipy库
<p>SciPy 是一个开源的 Python 库，用于科学和技术计算。它是基于 NumPy 构建的，提供了许多高级的数学、科学和工程模块。这些模块包括线性代数、优化、积分、插值、傅里叶变换、信号处理、图像处理、常微分方程求解以及其他任务所需的功能。</p>

### SciPy 的主要模块
* SciPy 包含许多模块，每个模块都提供了特定领域的计算功能：
	1.	scipy.linalg: 提供线性代数相关的操作，与 NumPy 的 linalg 模块相比，SciPy 的 linalg 模块提供了更多的高级函数，如矩阵分解、矩阵求逆、特征值求解等。
	2.	scipy.optimize: 用于函数的优化和求解，包括最小化、多项式拟合、根查找等。支持线性规划和非线性优化问题。
	3.	scipy.integrate: 提供函数的数值积分和微分方程的求解，包括一维和多维积分。
	4.	scipy.fft: 实现快速傅里叶变换（FFT）和反向变换，用于信号处理和分析。
	5.	scipy.signal: 提供信号处理工具，如滤波器设计、滤波、滤波器应用等。
	6.	scipy.stats: 提供概率分布和统计函数，用于描述性统计、检验统计、拟合分布等。
	7.	scipy.interpolate: 用于插值和样条拟合，包括一维和多维数据插值。
	8.	scipy.spatial: 提供空间数据结构和算法，如 KD 树、最近邻搜索、凸包计算等。
	9.	scipy.ndimage: 用于图像处理和分析，包括图像滤波、变换和特征检测。
	10.	scipy.io: 提供输入和输出功能，用于读取和写入 MATLAB、NetCDF、Fortran 等格式的数据文件。

### SciPy 的应用

* 科学研究：SciPy 提供的广泛功能使其成为科学研究的利器，特别是在需要复杂数学计算的领域，如物理、化学、生物学和工程学。
* 数据分析：SciPy 提供了丰富的数据分析工具，可以帮助用户进行数据处理、统计分析和建模。
* 机器学习和人工智能：结合其他库（如 NumPy、Pandas、Matplotlib、scikit-learn），SciPy 可以用来处理和分析数据集，为机器学习模型提供特征。
* 图像和信号处理：SciPy 的信号和图像处理模块为工程应用提供了强大的功能，适用于图像过滤、变换和信号分析。

### SciPy 的优势

* 高性能：SciPy 是基于 NumPy 的，并使用高效的 C 和 Fortran 代码实现，适合于大规模科学计算。
* 开源和活跃的社区：作为开源项目，SciPy 拥有活跃的开发者和用户社区，支持广泛的使用场景和最新的科学计算需求。
* 易于集成：SciPy 可以与其他科学计算库（如 Pandas、Matplotlib 等）无缝集成，提供全面的数据分析和可视化功能。

In [12]:
# -*- coding: utf-8 -*-
# 旧代码，tensorflow版本过旧，部分用例如session已弃用
import tensorflow as tf
import numpy as np
# scipy.io 和 scipy.misc：用于读取和保存图像数据
import scipy.io
import scipy.misc
import os
import time
from PIL import Image

# 打印当前的系统时间，用于记录训练过程的时间戳。便于计算耗时
def the_current_time():
	print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(time.time()))))

# 定义内容图像、风格图像、输出路径。
CONTENT_IMG = 'content.jpg'
STYLE_IMG = 'style5.jpg'
OUTPUT_DIR = 'neural_style_transfer_tensorflow/'

if not os.path.exists(OUTPUT_DIR):
	os.mkdir(OUTPUT_DIR)

# 定义输出图像的宽度、高度和通道数（RGB为3）。
IMAGE_W = 800
IMAGE_H = 600
COLOR_C = 3
# 定义噪声图像在初始输入图像中的权重（随机叠加的噪音层）、内容损失和风格损失函数的因子。
NOISE_RATIO = 0.7
BETA = 5
ALPHA = 100

VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'
# VGG19 预训练模型中使用的图像平均值，用于对输入图像进行预处理。
MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3))
'''
vgg = scipy.io.loadmat(VGG_MODEL)

vgg_layers = vgg['layers'] # 获取模型的所有层
print(vgg_layers[0][0][0][0][2][0][1])
'''

'''
创建一个 NumPy 数组，包含了三个通道的均值，这三个通道分别对应于图像的 RGB（红、绿、蓝）三个颜色通道。
	•	123.68, 116.779, 103.939：
	•	123.68 是蓝色通道的均值。
	•	116.779 是绿色通道的均值。
	•	103.939 是红色通道的均值。

这些均值是从 ImageNet 数据集计算得出的平均值，VGG19 网络在训练时使用了该数据集，因此对输入图像进行这样的均值调整是为了与网络训练时的输入保持一致。
	•	.reshape((1, 1, 1, 3))：将该数组重塑为形状 (1, 1, 1, 3)，这样可以在后续的计算中方便地与图像进行广播操作。
	•	1, 1, 1 是在对图像进行操作时加入的维度，以确保每个图像像素都能进行对应的减去均值操作。
	•	3 是颜色通道的数量，确保对 RGB 三个通道分别进行均值调整。
	'''

'\n创建一个 NumPy 数组，包含了三个通道的均值，这三个通道分别对应于图像的 RGB（红、绿、蓝）三个颜色通道。\n\t•\t123.68, 116.779, 103.939：\n\t•\t123.68 是蓝色通道的均值。\n\t•\t116.779 是绿色通道的均值。\n\t•\t103.939 是红色通道的均值。\n\n这些均值是从 ImageNet 数据集计算得出的平均值，VGG19 网络在训练时使用了该数据集，因此对输入图像进行这样的均值调整是为了与网络训练时的输入保持一致。\n\t•\t.reshape((1, 1, 1, 3))：将该数组重塑为形状 (1, 1, 1, 3)，这样可以在后续的计算中方便地与图像进行广播操作。\n\t•\t1, 1, 1 是在对图像进行操作时加入的维度，以确保每个图像像素都能进行对应的减去均值操作。\n\t•\t3 是颜色通道的数量，确保对 RGB 三个通道分别进行均值调整。\n\t'

## RELU修正线性单元
<p>ReLU（Rectified Linear Unit，修正线性单元）是一种常用的激活函数，广泛应用于深度学习模型中。ReLU 函数通过引入非线性，帮助神经网络学习复杂的数据模式。</p>

### ReLU 函数的定义
ReLU 是一种分段线性函数，定义如下：<br/>
 $\text{ReLU}(x) = \max(0, x) $ <br/>
这意味着 ReLU 函数在输入大于 0 时输出等于输入；输入小于或等于 0 时，输出为 0。

### 特性和优点

1.	非线性：虽然 ReLU 的形式看似简单（线性），但它引入了非线性性质，使神经网络能够拟合复杂的非线性函数。
2.	稀疏激活：ReLU 函数在输入小于或等于 0 时输出为 0，这种稀疏激活使得网络中的许多神经元在给定时间不会激活，从而提高了计算效率。
3.	梯度传播：相较于 Sigmoid 或 Tanh 等激活函数，ReLU 在正区间的梯度恒为 1，不会出现梯度消失的问题，这使得它在深层网络中更稳定。
4.	简单计算：ReLU 的计算量较小，仅需比较和取最大值，因此加快了训练速度。

### ReLU 的缺点

1.	死亡 ReLU 问题：在训练过程中，某些神经元可能永远不会被激活（即输入总是小于等于 0），导致这些神经元“死亡”，无法更新。
2.	不适用于所有数据：在某些情况下，ReLU 可能不适合处理数据的特定模式，尤其是含有负数特征的数据。

### ReLU 变体

为了克服 ReLU 的一些缺点，研究者提出了一些变体，如：
*	Leaky ReLU：在输入小于 0 时，输出为负斜率的线性函数。<br/>
 $\text{Leaky ReLU}(x) = \max(0.01x, x) $ <br/>

*	Parametric ReLU (PReLU)：在输入小于 0 时，输出为可学习的负斜率线性函数。<br/>
 $\text{PReLU}(x) = \max(ax, x) $ <br/>

	
*	Exponential Linear Unit (ELU)：在负区间有非线性变换。<br/>
 $\text{ELU}(x) = \begin{cases} x, & \text{if } x > 0 \\ \alpha (e^x - 1), & \text{if } x \leq 0 \end{cases} $ <br/>

 *	Scaled Exponential Linear Unit (SELU)：在负区间有非线性变换，且输出范围为 $(0, \infty)$。<br/>
 $\text{SELU}(x) = \lambda \text{ELU}(x) $ <br/>

## ReLU和层之间的关系
<p>在神经网络中，ReLU（Rectified Linear Unit）激活函数和层之间有着密切的关系。激活函数是网络层的重要组成部分，它决定了每个神经元的输出值。让我们详细探讨 ReLU 和网络层之间的关系。</p>

### 神经网络层的结构
一个典型的神经网络层包括以下几个部分：
1.	**输入**：上一层的输出或网络的初始输入数据。
2.	**权重和偏置**：每个神经元都有一组与输入连接的权重和一个偏置。
3.	**线性变换**：计算输入与权重的加权和，加上偏置。
4.	**激活函数**：应用于线性变换的输出，以引入非线性。
5.	**输出**：通过激活函数处理后的结果，作为下一层的输入。

### ReLU 在层中的作用

1. 引入非线性

	* 为什么需要非线性？：如果所有层之间都是线性变换（即仅仅是加权求和），那么不论网络有多少层，其效果等同于一层线性变换。因此，引入非线性（通过激活函数）是关键，它使得神经网络能够拟合复杂的非线性数据。
	* ReLU 的特点：ReLU 函数 f(x) = max(0, x) 引入了非线性，使得网络能够学习复杂的数据分布和模式。

2. 提高训练效率

	* 梯度传播：ReLU 在正区间的梯度恒为 1，因此在反向传播过程中不会出现梯度消失问题。这使得网络能够更快地收敛。
	* 稀疏激活：由于 ReLU 的输出在输入为负时为 0，这导致网络的许多神经元在给定时刻不激活，从而提高了网络的稀疏性和计算效率。


## VGG19架构
VGG19 是一种深度卷积神经网络（Convolutional Neural Network，CNN）架构，由牛津大学视觉几何组（Visual Geometry Group）在 2014 年提出。VGG19 是在 ImageNet 大规模视觉识别挑战赛（ILSVRC）中表现出色的模型之一，以其简洁且有效的网络设计而闻名。

### VGG19 的结构

VGG19 是 VGG 网络系列的一部分，其中数字 19 代表网络中的权重层（卷积层和全连接层）的总数。以下是 VGG19 网络的基本结构：

1.	卷积层（Convolutional Layers）：
	* 使用多个 3x3 的卷积核（filters）进行特征提取，步长为 1，填充为 same，即输出与输入的尺寸相同。
	* 卷积层之后接 ReLU 激活函数。
2.	池化层（Pooling Layers）：
	* 使用 2x2 的最大池化层（Max Pooling），步长为 2，减小特征图的尺寸。
3.	全连接层（Fully Connected Layers）：
	* 网络的最后包含三个全连接层，其中前两个每层有 4096 个节点，最后一层输出分类结果。
4.	输出层（Output Layer）：
	* 通过 Softmax 激活函数输出分类概率。

### VGG19 的具体配置
![jupyter](./VGG19.webp)
VGG19 的配置包括：

* 5 个卷积块（Convolutional Blocks），共 16 个卷积层：
* Block 1：2 个 3x3 卷积层 + Max Pooling
* Block 2：2 个 3x3 卷积层 + Max Pooling
* Block 3：4 个 3x3 卷积层 + Max Pooling
* Block 4：4 个 3x3 卷积层 + Max Pooling
* Block 5：4 个 3x3 卷积层 + Max Pooling
* 3 个全连接层：
* 2 个具有 4096 个神经元的全连接层
* 1 个具有 1000 个神经元的 Softmax 层（用于 ImageNet 1000 类别分类）

### VGG19 的特点

1.	深度和复杂性：通过增加层数（19 个权重层），VGG19 能够学习复杂的图像特征，适合处理复杂的视觉任务。
2.	小卷积核（3x3）：VGG19 使用 3x3 的卷积核，这种小卷积核的堆叠可以代替较大的卷积核，同时保留更细致的特征信息。
3.	大量参数：VGG19 拥有约 1.44 亿个参数，这使得它对计算资源要求较高。
4.	迁移学习：由于在 ImageNet 上的优秀表现，VGG19 经常被用作其他视觉任务的特征提取器，可以通过迁移学习应用于各种领域。

### 应用场景

* 图像分类：VGG19 可以用于多类图像分类任务。
* 特征提取：VGG19 的深度和结构使其适合用作特征提取器，提取高层次的特征用于其他任务。
* 风格迁移：VGG19 在神经风格迁移中被广泛使用，通过其卷积层提取内容和风格特征。

In [21]:

def load_vgg_model(path):
	'''
	VGG-19的每一个层都有一个int标识
	Details of the VGG19 model:
	- 0 is conv1_1 (3, 3, 3, 64)
	- 1 is relu
	- 2 is conv1_2 (3, 3, 64, 64)
	- 3 is relu    
	- 4 is maxpool
	- 5 is conv2_1 (3, 3, 64, 128)
	- 6 is relu
	- 7 is conv2_2 (3, 3, 128, 128)
	- 8 is relu
	- 9 is maxpool
	- 10 is conv3_1 (3, 3, 128, 256)
	- 11 is relu
	- 12 is conv3_2 (3, 3, 256, 256)
	- 13 is relu
	- 14 is conv3_3 (3, 3, 256, 256)
	- 15 is relu
	- 16 is conv3_4 (3, 3, 256, 256)
	- 17 is relu
	- 18 is maxpool
	- 19 is conv4_1 (3, 3, 256, 512)
	- 20 is relu
	- 21 is conv4_2 (3, 3, 512, 512)
	- 22 is relu
	- 23 is conv4_3 (3, 3, 512, 512)
	- 24 is relu
	- 25 is conv4_4 (3, 3, 512, 512)
	- 26 is relu
	- 27 is maxpool
	- 28 is conv5_1 (3, 3, 512, 512)
	- 29 is relu
	- 30 is conv5_2 (3, 3, 512, 512)
	- 31 is relu
	- 32 is conv5_3 (3, 3, 512, 512)
	- 33 is relu
	- 34 is conv5_4 (3, 3, 512, 512)
	- 35 is relu
	- 36 is maxpool
	- 37 is fullyconnected (7, 7, 512, 4096)
	- 38 is relu
	- 39 is fullyconnected (1, 1, 4096, 4096)
	- 40 is relu
	- 41 is fullyconnected (1, 1, 4096, 1000)
	- 42 is softmax
	'''
	vgg = scipy.io.loadmat(path)
	vgg_layers = vgg['layers'] # 获取模型的所有层
	
	'''
	•	scipy.io.loadmat(path)：loadmat 是 SciPy 库中的一个函数，用于从 .mat 文件中加载 MATLAB 格式的数据。.mat 文件是一种用于存储 MATLAB 数据的格式，可以包含多种类型的数据，包括矩阵、数组和结构体。
	•	参数 path：这是 .mat 文件的路径，文件中存储了预训练的 VGG19 模型的权重和架构。这个文件通常是通过从 ImageNet 数据集中训练得到的，并保存为 MATLAB 格式。
	•	vgg：loadmat 返回一个字典，vgg 是包含整个 .mat 文件数据的字典。这个字典的键对应于 .mat 文件中的变量名，值是对应变量的数据。
	'''
	# 提取指定layer[int]的权重W和偏置b，并验证层名是否正确
	def _weights(layer, expected_layer_name):
		W = vgg_layers[0][layer][0][0][2][0][0]
		b = vgg_layers[0][layer][0][0][2][0][1]
		layer_name = vgg_layers[0][layer][0][0][0][0]
		assert layer_name == expected_layer_name
		return W, b

	def _conv2d_relu(prev_layer, layer, layer_name):
		# 将 NumPy 数组 W 和 b 转换为 TensorFlow 常量：
		# W 是卷积核的权重矩阵，保持原有形状。
		# b 被重塑为一维张量，以便于在后续的运算中与输出特征图相加。
		W, b = _weights(layer, layer_name)
		W = tf.constant(W)
		b = tf.constant(np.reshape(b, (b.size)))
		'''
		tf.nn.conv2d：执行 2D 卷积操作。
			•	prev_layer：输入特征图。
			•	filter=W：卷积核（权重）。
			•	strides=[1, 1, 1, 1]：卷积步幅，1 表示每次移动一个像素。步幅的列表 [1, 1, 1, 1] 中，第一个和最后一个 1 是批处理大小和通道大小，通常保持不变。中间两个1表示特征图行、列的步幅。
			•	padding='SAME'：填充方式为“相同”，输出的特征图与输入的尺寸相同。
		tf.nn.relu：应用 ReLU 激活函数于卷积结果，计算公式为 max(0, x)，使网络具备非线性特征。
		'''
		# 在 2.x版本的tf.nn.conv2d 中，参数 filter 应为 filters。
		return tf.nn.relu(tf.nn.conv2d(prev_layer, filters=W, strides=[1, 1, 1, 1], padding='SAME') + b)
	
	# 函数 _avgpool 用于在 TensorFlow 中实现平均池化层（Average Pooling Layer）。池化层是卷积神经网络（CNN）中的一种下采样操作，用于减小特征图的尺寸、减少参数数量、控制过拟合以及增强模型的鲁棒性。
	# 平均池化（Average Pooling）：对每个池化窗口内的像素值求平均，结果作为输出特征图中的一个像素。
	# 下采样：通过池化操作，特征图的尺寸减少，通常能提取更具鲁棒性的特征，并提高模型的计算效率。
	# 鲁棒性（Robustness）是一个在计算机科学和工程领域中常用的术语，指系统或模型在面对不确定性、变化或噪声时，保持稳定和有效工作的能力。在机器学习和深度学习中，鲁棒性是指模型在数据变化或噪声干扰下仍能做出准确预测的能力。
	def _avgpool(prev_layer):
		return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
	
	# graph：存储 VGG19 网络各层输出的字典。逐层搭建 VGG19 网络，每层使用卷积加 ReLU 或池化操作。
	# 前五行代码解释：
	'''
	1. 初始化网络图
	graph = {}：创建一个空字典 graph，用于存储网络各层的输出张量。

	2. 定义输入层
	graph['input'] = tf.Variable(np.zeros((1, IMAGE_H, IMAGE_W, COLOR_C)), dtype='float32')：
		•	创建一个形状为 (1, IMAGE_H, IMAGE_W, COLOR_C) 的零张量，用作输入图像，初始化为全零。
		•	IMAGE_H、IMAGE_W：输入图像的高度和宽度。
		•	COLOR_C：输入图像的颜色通道数（如 RGB 图像为 3）。
		•	tf.Variable：将输入张量定义为 TensorFlow 变量，以便在训练过程中进行更新。

	3. 添加卷积层和激活函数
	graph['conv1_1'] = _conv2d_relu(graph['input'], 0, 'conv1_1')：
		•	使用 _conv2d_relu 函数在输入层上应用第一个卷积层，并紧跟 ReLU 激活函数。
		•	layer=0：表示从 VGG19 模型中提取 conv1_1 层的参数。
		•	layer_name='conv1_1'：验证层名称，确保从正确的层中提取参数。
	graph['conv1_2'] = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')：
		•	在第一个卷积层的输出上应用第二个卷积层，并紧跟 ReLU 激活函数。
		•	layer=2：表示从 VGG19 模型中提取 conv1_2 层的参数。
		•	layer_name='conv1_2'：验证层名称。

	4. 添加池化层
	graph['avgpool1'] = _avgpool(graph['conv1_2'])：
		•	在第二个卷积层的输出上应用平均池化操作，进行下采样。
		•	_avgpool：使用平均池化减少特征图的尺寸。
	'''
	graph = {}
	graph['input']    = tf.Variable(np.zeros((1, IMAGE_H, IMAGE_W, COLOR_C)), dtype='float32')
	graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
	graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
	graph['avgpool1'] = _avgpool(graph['conv1_2'])
	graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
	graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
	graph['avgpool2'] = _avgpool(graph['conv2_2'])
	graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
	graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
	graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
	graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
	graph['avgpool3'] = _avgpool(graph['conv3_4'])
	graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
	graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
	graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
	graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
	graph['avgpool4'] = _avgpool(graph['conv4_4'])
	graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
	graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
	graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
	graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
	graph['avgpool5'] = _avgpool(graph['conv5_4'])
	return graph

# 定义内容损失函数（Content Loss Function）：
def content_loss_func(content_features, generated_features):
	'''
	•	p：内容图像在特定层（如 conv4_2）的激活（特征图）。
	•	x：生成图像在相同层上的激活。
	•	N：特征图的通道数，即激活的深度。
	•	M：特征图的大小（即宽度乘以高度）。
	'''
	def _content_loss(p, x):
		N = p.shape[3]
		M = p.shape[1] * p.shape[2]
		# 使用均方误差（MSE）来计算内容损失，即生成图像和内容图像在某层特征图之间的误差平方和
		return (1 / (4 * N * M)) * tf.reduce_sum(tf.pow(x - p, 2))
	return _content_loss(content_features, generated_features) # 获取内容图像在 conv4_2 层的特征图,以及生成图并计算损失。

# STYLE_LAYERS定义了每个层在总风格损失中的贡献程度(权重)，每个层用一个元组表示，其中第一个元素是层的名称，第二个元素是该层的权重。
STYLE_LAYERS = [('conv1_1', 0.5), ('conv2_1', 1.0), ('conv3_1', 1.5), ('conv4_1', 3.0), ('conv5_1', 4.0)]
'''
旧用例的风格损失计算函数
def style_loss_func(style_features, generated_features):
	
	_gram_matrix：计算特征图的 Gram 矩阵，用于捕捉图像的风格信息。
	•	F：特征图，形状为 [1, height, width, channels]。
	•	N：特征图的通道数。
	•	M：特征图的空间大小，即高度乘以宽度。
	•	Ft = tf.reshape(F, (M, N))：将特征图重塑为二维矩阵，其中每一行代表一个通道的特征。
	•	tf.matmul(tf.transpose(Ft), Ft)：计算 Gram 矩阵，结果是通道与通道之间的内积矩阵，表示图像的风格特征。
	
	def _gram_matrix(F, N, M):
		Ft = tf.reshape(F, (M, N))
		return tf.matmul(tf.transpose(Ft), Ft)
	
	
	_style_loss：计算生成图像和风格图像在特定层的风格损失。
	•	a：风格图像在特定层的特征图
	•	x：生成图像在相同层的特征图
	
	def _style_loss(a, x): # a,x分别为风格图像及生成图像在特定层的特征图
		N = a.shape[3] # 特征图的通道数，表示每个特征图有多少个滤波器输出。
		M = a.shape[1] * a.shape[2] # 特征图的空间大小，即高度乘以宽度，表示每个通道内有多少个像素。
		A = _gram_matrix(a, N, M) # 特征图像的Gram矩阵，即风格特征
		G = _gram_matrix(x, N, M) # 生成图像的...
		# 使用均方误差（MSE）计算 Gram 矩阵之间的差异，并进行标准化。
		return (1 / (4 * N ** 2 * M ** 2)) * tf.reduce_sum(tf.pow(G - A, 2))

	return sum([_style_loss(model[layer_name], model[layer_name]) * w for layer_name, w in STYLE_LAYERS])
'''
# 新用例的风格损失函数
def style_loss_func(style_features, generated_features):
    def _gram_matrix(F):
        N = F.shape[3]
        M = F.shape[1] * F.shape[2]
        Ft = tf.reshape(F, (M, N))
        return tf.matmul(tf.transpose(Ft), Ft)

    A = _gram_matrix(style_features)
    G = _gram_matrix(generated_features)
    N = style_features.shape[3]
    M = style_features.shape[1] * style_features.shape[2]
    return (1 / (4 * (N ** 2) * (M ** 2))) * tf.reduce_sum(tf.square(G - A))

def generate_noise_image(content_image, noise_ratio=NOISE_RATIO):
	# 生成一个随机噪声图像，像素值在 -20 到 20 之间，形状为 (1, IMAGE_H, IMAGE_W, COLOR_C)，并叠加于内容图像生成初始图像
	noise_image = np.random.uniform(-20, 20, (1, IMAGE_H, IMAGE_W, COLOR_C)).astype('float32')
	input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio) # 这个组合可以理解为在噪声和内容之间的平衡：噪声比例越高，初始图像的随机性越大；噪声比例越低，初始图像越接近原始内容图像。
	return tf.Variable(input_image, dtype=tf.float32)

'''
	初始输入图像：神经风格迁移的优化过程从该初始图像开始，通过反复迭代修改以最小化损失函数（内容损失和风格损失的加权和），最终得到融合了内容和风格特征的生成图像。
	探索解空间：通过引入噪声，优化过程可以探索更广泛的解空间，避免过早陷入局部最优解。噪声的引入有助于生成图像突破内容图像的限制，向目标风格图像的方向进行调整。
	控制生成过程：通过调整 noise_ratio，可以控制生成图像在内容和风格之间的偏好。例如，较低的噪声比例通常使初始图像更贴近内容图像，这可能有助于更快地收敛到内容一致的结果。
'''

'''
由于scipy.misc已被弃用，改用使用PIL.Image实现
def load_image(path):
	image = scipy.misc.imread(path) # 读取图像文件，路径由参数 path 指定。
	image = scipy.misc.imresize(image, (IMAGE_H, IMAGE_W)) # 将图像调整为指定大小 (IMAGE_H, IMAGE_W)。
	image = np.reshape(image, ((1, ) + image.shape))  # 将图像重塑为 4D 张量 [1, height, width, channels]，以适应神经网络的输入格式。
	image = image - MEAN_VALUES # 减去训练 VGG 网络时使用的均值（MEAN_VALUES），以进行数据标准化。这是预处理步骤，使图像输入符合网络的训练条件。
	return image

def save_image(path, image):
	image = image + MEAN_VALUES # 加回均值以恢复图像的原始颜色范围，逆转预处理步骤。
	image = image[0] # 移除批量维度，将 4D 张量恢复为 3D 图像数据。
	image = np.clip(image, 0, 255).astype('uint8') # 将像素值限制在 0 到 255 之间，并转换为 uint8 类型，以便保存为标准图像格式。
	scipy.misc.imsave(path, image) # 保存图像到指定路径 path。
'''
def load_image(path):
    image = Image.open(path)
    image = image.resize((IMAGE_W, IMAGE_H))
    image = np.array(image)
    image = np.reshape(image, ((1,) + image.shape))
    image = image - MEAN_VALUES
    return image

def save_image(path, image):
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    image = Image.fromarray(image)
    image.save(path)

the_current_time()

2024-08-11 20:40:43


In [20]:
# Session弃用——旧代码
with tf.Session() as sess: # 创建一个 TensorFlow 会话，用于执行计算图。
	content_image = load_image(CONTENT_IMG)
	style_image = load_image(STYLE_IMG)
	model = load_vgg_model(VGG_MODEL)

	input_image = generate_noise_image(content_image)
	sess.run(tf.global_variables_initializer()) # 初始化 TensorFlow 图中的所有变量。（2.x弃用，自动初始化变量）

	sess.run(model['input'].assign(content_image)) # 将内容图像赋给模型的输入层。
	content_loss = content_loss_func(sess, model)

	sess.run(model['input'].assign(style_image)) # 将风格图像赋给模型的输入层。
	style_loss = style_loss_func(sess, model)

	total_loss = BETA * content_loss + ALPHA * style_loss # 计算总损失，将内容损失和风格损失结合起来，BETA 和 ALPHA 是权重系数。
	optimizer = tf.train.AdamOptimizer(2.0) # 创建 Adam 优化器，用于更新生成图像。
	train = optimizer.minimize(total_loss) # 定义训练操作以最小化总损失。

	sess.run(tf.global_variables_initializer()) # 再次初始化所有变量。
	sess.run(model['input'].assign(input_image)) # 将初始输入图像赋给模型的输入层。


	ITERATIONS = 2000 # 设置迭代次数。
	for i in range(ITERATIONS):
		sess.run(train) # 执行一次训练步骤，更新生成图像以最小化总损失。
		if i % 100 == 0:
			# 每 100 次迭代：
			output_image = sess.run(model['input']) # 获取当前生成图像。
			the_current_time()
			print('Iteration %d' % i)
			print('Cost: ', sess.run(total_loss)) # 打印当前迭代次数和损失值。

			save_image(os.path.join(OUTPUT_DIR, 'output_%d.jpg' % i), output_image)

AttributeError: module 'tensorflow' has no attribute 'Session'

### AttributeError: module 'tensorflow' has no attribute 'Session'
* 错误通常发生在使用 TensorFlow 2.x 版本时。TensorFlow 2.x 引入了许多 API 的变化，其中一个重要变化是使用了简化的即刻执行（eager execution）模式，不再需要显式创建会话（Session）来运行计算图。
* 要解决这个问题，需要将代码修改为使用 TensorFlow 2.x 的新 API。这包括利用 tf.function 装饰器以及直接调用计算图的函数接口。下面是如何更新代码以适应 TensorFlow 2.x：

**更新代码**

* 以下是如何调整你的代码以适应 TensorFlow 2.x 的方法：

	1.	使用即刻执行：TensorFlow 2.x 默认启用即刻执行，因此无需显式创建和运行会话。
	2.	使用 tf.function：将计算图转换为函数。
	3.	移除 sess.run：直接操作 TensorFlow 变量和计算。

In [24]:
def compute_loss(model, content_image, style_image, generated_image):
    # 提取特征
    model['input'].assign(content_image)
    content_features = model['conv4_2'].numpy()

    model['input'].assign(style_image)
    style_features = [model[layer].numpy() for layer, _ in STYLE_LAYERS]

    model['input'].assign(generated_image)
    generated_features = [model[layer] for layer, _ in STYLE_LAYERS]

    # 计算内容损失
    content_loss = content_loss_func(content_features, model['conv4_2'])

    # 计算风格损失
    style_loss = sum(style_loss_func(style_features[i], generated_features[i]) * weight for i, (_, weight) in enumerate(STYLE_LAYERS))

    # 计算总损失
    total_loss = BETA * content_loss + ALPHA * style_loss
    return total_loss

def optimize_image(content_image, style_image, model, iterations=2000):
    generated_image = generate_noise_image(content_image)
    optimizer = tf.keras.optimizers.Adam(learning_rate=2.0)

    for i in range(iterations):
        with tf.GradientTape() as tape: # 用于记录前向传播过程中计算图的操作，方便后续计算梯度。在 with 语句中，计算损失 total_loss。
            tape.watch(generated_image)
            total_loss = compute_loss(model, content_image, style_image, generated_image)
      
        grads = tape.gradient(total_loss, generated_image) # 计算总损失相对于生成图像的梯度。
        optimizer.apply_gradients([(grads, generated_image)]) # 使用计算得到的梯度更新生成图像。通过梯度下降法调整生成图像的像素值以最小化总损失。
        if i % 100 == 0:
            output_image = generated_image.numpy()
            print('Iteration %d' % i)
            print('Cost: ', total_loss.numpy())
            save_image(os.path.join(OUTPUT_DIR, 'output_%d.jpg' % i), output_image)
def main():
    content_image = load_image(CONTENT_IMG)
    style_image = load_image(STYLE_IMG)
    model = load_vgg_model(VGG_MODEL)
    optimize_image(content_image, style_image, model)

if __name__ == "__main__":
    main()



ValueError: No gradients provided for any variable.

In [45]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import scipy.io
import os
import time
from PIL import Image

# 打印当前的系统时间，用于记录训练过程的时间戳。便于计算耗时
def the_current_time():
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(time.time()))))

# 定义内容图像、风格图像、输出路径。
CONTENT_IMG = 'content.jpg'
STYLE_IMG = 'style5.jpg'
OUTPUT_DIR = 'neural_style_transfer_tensorflow/'

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

# 定义输出图像的宽度、高度和通道数（RGB为3）。
IMAGE_W = 800
IMAGE_H = 600
COLOR_C = 3
# 定义噪声图像在初始输入图像中的权重（随机叠加的噪音层）、内容损失和风格损失函数的因子。
NOISE_RATIO = 0.7
BETA = 5
ALPHA = 100

VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'
# VGG19 预训练模型中使用的图像平均值，用于对输入图像进行预处理。
MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3)).astype('float32')

def load_vgg_model(path):
    vgg = scipy.io.loadmat(path)
    vgg_layers = vgg['layers']

    def _weights(layer, expected_layer_name):
        W = vgg_layers[0][layer][0][0][2][0][0]
        b = vgg_layers[0][layer][0][0][2][0][1]
        layer_name = vgg_layers[0][layer][0][0][0][0]
        assert layer_name == expected_layer_name
        return W, b

    def _conv2d_relu(prev_layer, layer, layer_name):
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.relu(tf.nn.conv2d(prev_layer, filters=W, strides=[1, 1, 1, 1], padding='SAME') + b)

    def _avgpool(prev_layer):
        return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    graph = {}
    graph['input']    = tf.Variable(np.zeros((1, IMAGE_H, IMAGE_W, COLOR_C)), dtype='float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    return graph

# 定义内容损失函数（Content Loss Function）：
def content_loss_func(content_features, generated_features):
    def _content_loss(p, x):
        N = p.shape[3]
        M = p.shape[1] * p.shape[2]
        return (1 / (4 * N * M)) * tf.reduce_sum(tf.pow(x - p, 2))
    return _content_loss(content_features, generated_features)

# STYLE_LAYERS定义了每个层在总风格损失中的贡献程度(权重)，每个层用一个元组表示，其中第一个元素是层的名称，第二个元素是该层的权重。
STYLE_LAYERS = [('conv1_1', 0.5), ('conv2_1', 1.0), ('conv3_1', 1.5), ('conv4_1', 3.0), ('conv5_1', 4.0)]

# 新用例的风格损失函数
def style_loss_func(style_features, generated_features):
    def _gram_matrix(F):
        N = F.shape[3]
        M = F.shape[1] * F.shape[2]
        Ft = tf.reshape(F, (M, N))
        return tf.matmul(tf.transpose(Ft), Ft)

    A = _gram_matrix(style_features)
    G = _gram_matrix(generated_features)
    N = style_features.shape[3]
    M = style_features.shape[1] * style_features.shape[2]
    return (1 / (4 * (N ** 2) * (M ** 2))) * tf.reduce_sum(tf.square(G - A))

def generate_noise_image(content_image, noise_ratio=NOISE_RATIO):
    noise_image = np.random.uniform(-20, 20, (1, IMAGE_H, IMAGE_W, COLOR_C)).astype('float32')
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    return tf.Variable(input_image, dtype=tf.float32, trainable=True)

def load_image(path):
    image = Image.open(path)
    image = image.resize((IMAGE_W, IMAGE_H))
    image = np.array(image).astype('float32')
    image = np.reshape(image, ((1,) + image.shape))
    image = image - MEAN_VALUES
    return image

def save_image(path, image):
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    image = Image.fromarray(image)
    image.save(path)

the_current_time()

def compute_loss(model, content_image, style_image, generated_image):
    model['input'].assign(content_image)
    content_features = model['conv4_2']

    model['input'].assign(style_image)
    style_features = [model[layer] for layer, _ in STYLE_LAYERS]

    model['input'].assign(generated_image)
    generated_features = [model[layer] for layer, _ in STYLE_LAYERS]

    content_loss = content_loss_func(content_features, model['conv4_2'])
    style_loss = sum(style_loss_func(style_features[i], generated_features[i]) * weight 
                     for i, (_, weight) in enumerate(STYLE_LAYERS))
    total_loss = BETA * content_loss + ALPHA * style_loss
    return total_loss

def optimize_image(content_image, style_image, model, iterations=2000):
    generated_image = generate_noise_image(content_image)
    optimizer = tf.keras.optimizers.Adam(learning_rate=2.0)

    for i in range(iterations):
        with tf.GradientTape() as tape:
            tape.watch(generated_image)
            total_loss = compute_loss(model, content_image, style_image, generated_image)
        grads = tape.gradient(total_loss, generated_image)
        if grads is None:
            raise ValueError("梯度为空。请检查模型和图形连接。")
        
        optimizer.apply_gradients([(grads, generated_image)])
        
        if i % 100 == 0:
            output_image = generated_image.numpy()
            print(f'Iteration {i}, Cost: {total_loss.numpy()}')
            save_image(os.path.join(OUTPUT_DIR, f'output_{i}.jpg'), output_image)

def main():
    content_image = load_image(CONTENT_IMG)
    style_image = load_image(STYLE_IMG)
    model = load_vgg_model(VGG_MODEL)
    optimize_image(content_image, style_image, model)

if __name__ == "__main__":
    main()


2024-08-12 02:00:06


ValueError: 梯度为空。请检查模型和图形连接。